# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)
#mouse_metadata
#study_results
# Combine the data into a single DataFrame
merged_data = pd.merge(study_results, mouse_metadata, how="left", on = "Mouse ID")

# Display the data table for preview
merged_data.head() 

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [2]:
# Checking the number of mice.
mice_data = merged_data["Mouse ID"].unique()
number_of_mice = len(mice_data)
number_of_mice

249

In [3]:
# Our data should be uniquely identified by Mouse ID and Timepoint
duplicate_rows = merged_data[merged_data.duplicated(subset=["Mouse ID", "Timepoint"])] 
duplicate_rows

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
137,g989,0,45.000000,0,Propriva,Female,21,26
360,g989,5,47.570392,0,Propriva,Female,21,26
681,g989,10,49.880528,0,Propriva,Female,21,26
869,g989,15,53.442020,0,Propriva,Female,21,26
1111,g989,20,54.657650,1,Propriva,Female,21,26


In [ ]:
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mice_dupes = merged_data[merged_data[["Mouse ID", "Timepoint"]].duplicated() == True]
mice_dupes_id = mice_dupes["Mouse ID"].unique()
mice_dupes_id

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
mice_dupes_data = merged_data.loc[merged_data["Mouse ID"] == mice_dupes_id[0]]
mice_dupes_data

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = merged_data.drop_duplicates(subset=["Mouse ID", "Timepoint"]) 
clean_df.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
num_mice = merged_data["Mouse ID"].nunique() 
num_mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
regimen_group = merged_data.groupby("Drug Regimen")
regimen_stats = regimen_group["Tumor Volume (mm3)"].agg(["mean", "median", "var", "std", "sem"])
regimen_stats.columns = ["Mean Tumor Volume", "Median Tumor Volume", "Tumor Volume Variance", "Tumor Volume Std. Dev.", "Tumor Volume Std. Err."]
regimen_stats

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
regimen_stats = regimen_group["Tumor Volume (mm3)"].agg(["mean", "median", "var", "std", "sem"])
regimen_stats

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
# Count the whole number of rows for each drug regimen.
regimen_counts = merged_data["Drug Regimen"].value_counts()
regimen_counts
# Set a bar plot of the regimen counts using Pandas.
regimen_counts.plot(kind="bar", figsize=(10,6), rot=45)
regimen_counts
#Create labels for the X and Y.
plt.xticks(rotation="vertical")
plt.xlabel("Drug Regimen", color='Red')
plt.ylabel("# of Observed Mouse Timepoints", color='Red')


In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
# Count the whole number of rows for each drug regimen.
regimen_counts = merged_data["Drug Regimen"].value_counts()
regimen_counts
# Set a bar plot of the regimen counts using pyplot.
plt.bar(regimen_counts.index, regimen_counts.values) 
plt.bar
#Create labels for the X and Y.
plt.xticks(rotation="vertical")
plt.xlabel("Drug Regimen", color='Red')
plt.ylabel("# of Observed Mouse Timepoints", color='Red')


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
#Find the number of female and male mice.
male_v_female = merged_data["Sex"].value_counts()
# Set a pie plot of the sex counts using Pandas
male_v_female.plot.pie(figsize=(5, 5), autopct="%1.1f%%")
plt.show

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
#Find the number of female and male mice.
male_v_female = merged_data["Sex"].value_counts()
plt.ylabel("Sex")

# Set a pie plot of the sex counts using pyplot
plt.pie(male_v_female, labels = male_v_female.index, autopct="%1.1f%%")
male_v_female
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
relevant_treatments_df = merged_data[merged_data["Drug Regimen"].isin(["Capomulin", "Ramicane", "Infubinol", "Ceftamin"])]
relevant_treatments_df

# Start by getting the last (greatest) timepoint for each mouse
last_timepoint_df = merged_data.groupby("Mouse ID")["Timepoint"].max()
last_timepoint_df

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
last_timepoint_merged_data = pd.merge(merged_data, last_timepoint_df, on=["Mouse ID", "Timepoint"], how="inner")
last_timepoint_merged_data

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatments = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]

# Create empty list to fill with tumor vol data (for plotting)
tum_vol = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatments:
    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    final_tum_vol = merged_data.loc[merged_data["Drug Regimen"] == drug, 'Tumor Volume (mm3)']
    
    # add subset 
    tum_vol.append(final_tum_vol)
    
    # Determine outliers using upper and lower bounds
    quartiles = final_tum_vol.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    outliers = final_tum_vol.loc[(final_tum_vol < lower_bound) | (final_tum_vol > upper_bound)]
    print(f"{drug}'s potential outliers: {outliers}")
    
    

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.

orange_out = dict(markerfacecolor='red',markersize=10)
plt.boxplot(tum_vol, labels = treatments,flierprops=orange_out)
plt.ylabel('Tumor Volume (mm3)')
plt.show()




## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
mouse_id = 'r944'
drug_regimen = 'Capomulin'
drug_regimen

#Set tumor volume vs. time point
subset = merged_data.loc[(merged_data['Mouse ID'] == mouse_id) & (merged_data['Drug Regimen'] == drug_regimen), ['Timepoint', 'Tumor Volume (mm3)']]
subset

#Plot the values
fig, ax = plt.subplots()
ax.plot(subset['Timepoint'], subset['Tumor Volume (mm3)'], marker='o') 

# Set the x-axis and y-axis labels and title

ax.set_xlabel('Timepoint', color='Red')
ax.set_ylabel('Tumor Volume (mm3)', color='Red')
ax.set_title(f"Tumor Volume vs.mouse {mouse_id} Treated with {drug_regimen}", color='Red')



In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
capomulin_df = merged_data[merged_data['Drug Regimen'] == 'Capomulin']
capomulin_df
#Find the average tumor volume for each mouse 
avg_tumor_vol_df = capomulin_df.groupby(['Mouse ID'])['Tumor Volume (mm3)'].mean()
avg_tumor_vol_df

# Create a scatter plot of mouse weight vs. average tumor volume
plt.scatter(merged_data['Weight (g)'], merged_data['Tumor Volume (mm3)']) 

# Set the chart title and axis labels
plt.title('Mouse Weight vs. Tumor Volume (mm3 Capomulin Regimen)', color='Red')
plt.xlabel('Weight (g)', color='Red')
plt.ylabel('Average Tumor Volume (mm3)', color='Red')

# Show the plot
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen

# take the weight with average tumor volume data for Capomulin regimen
capomulin_data = merged_data[merged_data['Drug Regimen'] == 'Capomulin']
capomulin_data = capomulin_data.groupby('Mouse ID').agg({'Weight (g)': 'mean', 'Tumor Volume (mm3)': 'mean'})
capomulin_data

# Set a scatter plot of mouse weight vs. average observed tumor volume
fig, ax = plt.subplots()
ax.scatter(capomulin_data['Weight (g)'], capomulin_data['Tumor Volume (mm3)'])
ax.set_xlabel('Mouse Weight (g)', color='Red')
ax.set_ylabel('Average Tumor Volume (mm3)', color='Red')
ax.set_title('Mouse Weight vs. Average Tumor Volume for Capomulin Regimen', color='Red')

# Calculate the correlation coefficient

correlation_coefficient= st.pearsonr(capomulin_data['Weight (g)'], capomulin_data['Tumor Volume (mm3)'])
print(f"The correlation between Mouse weight (g) and Average Tumor Volume (mm3) is {round(correlation_coefficient[0],3)}")

# Plot the linear regression model
slope, intercept, r_value, p_value, std_err = st.linregress(capomulin_data['Weight (g)'], capomulin_data['Tumor Volume (mm3)'])
regress_values = slope * capomulin_data['Weight (g)'] + intercept
regress_values

# Create the linear regression line to the scatter plot
ax.plot(capomulin_data['Weight (g)'], regress_values, color='Green')

plt.show()